In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os

## shachar
dirPath = "/content/drive/MyDrive/Colab Notebooks/Final project/Experiments"
## ariel
# dirPath = "/content/drive/MyDrive/Colab Notebooks/Experiments"

os.chdir(dirPath)

In [3]:
!pip install tensorflow-addons

In [4]:
from Data_extraction_transformer import Get_Data
from Results import Get_Results

import collections
import logging
import os
import pathlib
import re
import string
import sys
import time
import math

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow import keras
from keras import backend as K
import matplotlib.pyplot as plt

In [5]:
K_SEED = 330

class args:

  def __init__(self,input_data,roi,net,roi_name,zscore,train_size):
    self.input_data = input_data
    self.roi = roi
    self.net = net
    self.roi_name = roi_name
    self.K_RUNS = K_RUNS
    # preprocessing
    self.zscore = zscore
    # training parameters
    self.train_size = train_size

# data parameters
args.input_data = 'data/roi_ts'
args.roi = 300
args.net = 7
args.roi_name = 'roi'
args.K_RUNS = 4
# preprocessing
args.zscore = 1
# training parameters
args.train_size = 100

In [6]:
def _get_clip_labels():
    '''
    assign all clips within runs a label
    use 0 for testretest
    '''
    # where are the clips within the run?
    timing_file = pd.read_csv('data/videoclip_tr_lookup.csv')

    clips = []
    for run in range(args.K_RUNS):
        run_name = 'MOVIE%d' %(run+1) #MOVIEx_7T_yz
        timing_df = timing_file[timing_file['run'].str.contains(run_name)]  
        timing_df = timing_df.reset_index(drop=True)

        for jj, row in timing_df.iterrows():
            clips.append(row['clip_name'])
            
    clip_y = {}
    jj = 1
    for clip in clips:
        if 'testretest' in clip:
            clip_y[clip] = 0
        else:
            clip_y[clip] = jj
            jj += 1

    return clip_y

clip_y = _get_clip_labels()
k_class = len(np.unique(list(clip_y.values())))
print('number of classes = %d' %k_class)

clip_names = np.zeros(k_class).astype(str)
clip_names[0] = 'testretest'
for key, item in clip_y.items():
    if item!=0:
        clip_names[item] = key

number of classes = 15


In [7]:
X_train, train_len, y_train, X_val, val_len, y_val, X_test, test_len, y_test, train_list, test_list, clip_time = Get_Data(args)

---
roi
---
---
roi
---
loading run 1/4
loading run 2/4
loading run 3/4
loading run 4/4
data loading time: 162.63 seconds
number of subjects = 176
number of features = 300
number of classes = 15
seq lengths = [ 84 245 222 189  65 227 260 250 181 186 205 143 233 231 256]


In [8]:
def GetNetwork(X_train, X_val, X_test,startLH,endLH,startRH,endRH):
  X_train_LH = X_train[:,:,startLH:endLH]
  X_train_RH = X_train[:,:,startRH:endRH]
  X_train_end = tf.concat([X_train_LH, X_train_RH], axis=2)

  X_val_LH = X_val[:,:,startLH:endLH]
  X_val_RH = X_val[:,:,startRH:endRH]
  X_val_end = tf.concat([X_val_LH, X_val_RH], axis=2)

  X_test_LH = X_test[:,:,startLH:endLH]
  X_test_RH = X_test[:,:,startRH:endRH]
  X_test_end = tf.concat([X_test_LH, X_test_RH], axis=2)

  return X_train_end, X_val_end, X_test_end

networksDict = {'vis':{'startLH':0,'endLH':24,'startRH':151,'endRH':174,'train':[], 'val':[], 'test':[]},
                'SomMot':{'startLH':24,'endLH':53,'startRH':174,'endRH':201,'train':[], 'val':[], 'test':[]},
                'Attn':{'startLH':53,'endLH':85,'startRH':201,'endRH':237,'train':[], 'val':[], 'test':[]},
                'limbic':{'startLH':85,'endLH':95,'startRH':237,'endRH':247,'train':[], 'val':[], 'test':[]},
                'Cont':{'startLH':95,'endLH':112,'startRH':247,'endRH':270,'train':[], 'val':[], 'test':[]},
                'DMN':{'startLH':112,'endLH':150,'startRH':270,'endRH':300,'train':[], 'val':[], 'test':[]},
                'full':{'train':X_train, 'val':X_val, 'test':X_test},
                'fullShuffled':{'train':X_train, 'val':X_val, 'test':X_test},
                'fullOffset':{'train':X_train, 'val':X_val, 'test':X_test}
                }

for net in networksDict:
  if net == 'full' or net == 'fullShuffled' or net == 'fullOffset': continue
  networksDict[net]['train'], networksDict[net]['val'], networksDict[net]['test'] = GetNetwork(X_train, X_val, X_test, networksDict[net]['startLH'], networksDict[net]['endLH'], networksDict[net]['startRH'], networksDict[net]['endRH'])
  print(net + ':')
  print('train: '+ str(networksDict[net]['train'].shape))
  print('val: '+ str(networksDict[net]['val'].shape))
  print('test: '+ str(networksDict[net]['test'].shape))

vis:
train: (1800, 260, 47)
val: (684, 260, 47)
test: (684, 260, 47)
SomMot:
train: (1800, 260, 56)
val: (684, 260, 56)
test: (684, 260, 56)
Attn:
train: (1800, 260, 68)
val: (684, 260, 68)
test: (684, 260, 68)
limbic:
train: (1800, 260, 20)
val: (684, 260, 20)
test: (684, 260, 20)
Cont:
train: (1800, 260, 40)
val: (684, 260, 40)
test: (684, 260, 40)
DMN:
train: (1800, 260, 68)
val: (684, 260, 68)
test: (684, 260, 68)


Encoder build

In [9]:
from tensorflow.keras.layers import MultiHeadAttention

def lr_scheduler(epoch, lr, warmup_epochs=13, decay_epochs=100, initial_lr=1e-6, base_lr=1e-4, min_lr=5e-5):
    if epoch <= warmup_epochs:
        pct = epoch / warmup_epochs
        return ((base_lr - initial_lr) * pct) + initial_lr

    if epoch > warmup_epochs and epoch < warmup_epochs+decay_epochs:
        pct = 1 - ((epoch - warmup_epochs) / decay_epochs)
        return ((base_lr - min_lr) * pct) + min_lr

    return min_lr

class Time2Vec(keras.layers.Layer):
    def __init__(self, kernel_size=1):
        super(Time2Vec, self).__init__(trainable=True, name='Time2VecLayer')
        self.k = kernel_size
    
    def build(self, input_shape):
        # trend
        self.wb = self.add_weight(name='wb',shape=(input_shape[1],),initializer='uniform',trainable=True)
        self.bb = self.add_weight(name='bb',shape=(input_shape[1],),initializer='uniform',trainable=True)
        # periodic
        self.wa = self.add_weight(name='wa',shape=(1, input_shape[1], self.k),initializer='uniform',trainable=True)
        self.ba = self.add_weight(name='ba',shape=(1, input_shape[1], self.k),initializer='uniform',trainable=True)
        super(Time2Vec, self).build(input_shape)
    
    def call(self, inputs, **kwargs):
        bias = self.wb * inputs + self.bb
        dp = K.dot(inputs, self.wa) + self.ba
        wgts = K.sin(dp) # or K.cos(.)

        ret = K.concatenate([K.expand_dims(bias, -1), wgts], -1)
        ret = K.reshape(ret, (-1, inputs.shape[1]*(self.k+1)))
        return ret
    
    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[1]*(self.k + 1))

class AttentionBlock(keras.Model):
    def __init__(self, name='AttentionBlock', num_heads=2, head_size=128, ff_dim=None, dropout=0, **kwargs):
        super().__init__(name=name, **kwargs)

        if ff_dim is None:
            ff_dim = head_size

        self.attention = MultiHeadAttention(num_heads=num_heads, key_dim=head_size, dropout=dropout)
        self.attention_dropout = keras.layers.Dropout(dropout)
        self.attention_norm = keras.layers.LayerNormalization(epsilon=1e-6)

        self.ff_conv1 = keras.layers.Conv1D(filters=ff_dim, kernel_size=1, activation='relu')
        # self.ff_conv2 at build()
        self.ff_dropout = keras.layers.Dropout(dropout)
        self.ff_norm = keras.layers.LayerNormalization(epsilon=1e-6)

    def build(self, input_shape):
        self.ff_conv2 = keras.layers.Conv1D(filters=input_shape[-1], kernel_size=1) 

    def call(self, inputs):
        x, attention_scores = self.attention(inputs, inputs, return_attention_scores=True)
        x = self.attention_dropout(x)
        x = self.attention_norm(inputs + x)

        x = self.ff_conv1(x)
        x = self.ff_conv2(x)
        x = self.ff_dropout(x)

        x = self.ff_norm(inputs + x)
        return x, attention_scores

class ModelTrunk(keras.Model):
      def __init__(self, classes, inputs, name='ModelTrunk', time2vec_dim=1, num_heads=2, head_size=128, ff_dim=None, num_layers=1, dropout=0, **kwargs):
        super().__init__(name=name, **kwargs)
        self.MaskingLayer = keras.layers.Masking(mask_value=0.0, input_shape = [None, inputs.shape[-1]])
        self.time2vec = Time2Vec(kernel_size=time2vec_dim)
        if ff_dim is None:
            ff_dim = head_size
        self.dropout = dropout
        self.classes = classes
        self.attention_layers = [AttentionBlock(num_heads=num_heads, head_size=head_size, ff_dim=ff_dim, dropout=dropout) for _ in range(num_layers)]
        self.dense_layer = keras.layers.Dense(units = 512, activation = 'relu')
        self.dropout_layer = keras.layers.Dropout(dropout)
        self.final_layer = tf.keras.layers.Dense(classes, activation='softmax')

        
      def call(self, inputs):
        #time_embedding = keras.layers.TimeDistributed(self.time2vec)(inputs)
        #x = K.concatenate([inputs, time_embedding], -1)
        #x = MaskingLayer(inputs)
        x = inputs
        for attention_layer in self.attention_layers:
            x, attention_scores = attention_layer(x)
        x = self.dense_layer(x)
        x = self.dropout_layer(x)
        x = self.final_layer(x)

        return x

In [14]:
EPOCHS = 45
num_layers = [4, 6, 8]
d_model = 300
dff = 260
key_dim = 260
num_heads = [1, 4, 6, 8]
dropout_rate = 0.1
BUFFER_SIZE = 20000
BATCH_SIZE = 64

In [ ]:
import pickle
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay

## vis: for heads:[6 8] missing layers: 4
## limbic for heads: 8 missing layers: 4
## DMN: for heads: 8 missing layers: [4 6]

def make_batches(ds):
  return (
      ds
      .cache()
      .shuffle(BUFFER_SIZE)
      .batch(BATCH_SIZE)
      .prefetch(tf.data.AUTOTUNE))

def GetCmat(model, X, y):
  y_hat_hold = model.predict(X)
  y_hat = np.argmax(y_hat_hold, axis=2)
  true_y = y

  y_overtime = []
  y_hat_overtime = []
  for rows_y,rows_y_hat in zip(true_y,y_hat):
    values, counts = np.unique(rows_y, return_counts=True)
    ind = np.argmax(counts)
    if values[ind] == 15:
      counts[ind] = 0
      ind = np.argmax(counts)
    y_overtime.append(values[ind])

    values, counts = np.unique(rows_y_hat, return_counts=True)
    ind = np.argmax(counts)
    if values[ind] == 15:
      counts[ind] = 0
      ind = np.argmax(counts)
    y_hat_overtime.append(values[ind])

  cm = confusion_matrix(y_overtime,y_hat_overtime)
  acc = accuracy_score(y_overtime,y_hat_overtime)
  return cm, acc

def printLoop(net, head, layer):
  print('-----------------------------------------------------------------------------------------------------')
  print('-----------------------------------------------------------------------------------------------------')
  print('-----------------------------------------------------------------------------------------------------')
  print(f'---------------------------NET: {net}  number of heads: {head} number of layers: {layer}--------------------------')
  print('-----------------------------------------------------------------------------------------------------')
  print('-----------------------------------------------------------------------------------------------------')
  print('-----------------------------------------------------------------------------------------------------')


for net in networksDict:
  if net == 'fullOffset' or net == 'fullShuffled': continue
  for heads in num_heads:
    for layers in num_layers:
      printLoop(net, heads, layers)
      train_data = tf.data.Dataset.from_tensor_slices((networksDict[net]['train'],y_train))
      val_data = tf.data.Dataset.from_tensor_slices((networksDict[net]['val'],y_val))
      input = networksDict[net]['train']
      train_batch = make_batches(train_data)
      val_batch = make_batches(val_data)

      myModel = ModelTrunk(inputs=input, name=f'Model_{net}_numHeads_{heads}', time2vec_dim=1, num_heads=heads, head_size=key_dim, ff_dim=dff, num_layers=layers, dropout=dropout_rate, classes=(len(clip_time)+1))

      myModel.compile(
          optimizer=keras.optimizers.Adam(learning_rate=1e-4),
          loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
          metrics=[keras.metrics.SparseCategoricalAccuracy()]
      )

      callbacks = keras.callbacks.LearningRateScheduler(lr_scheduler, verbose=0)

      # EarlyStopping criteria
      early_stopping = keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)

      history = myModel.fit(train_batch, validation_data=val_batch, batch_size=BATCH_SIZE, epochs=EPOCHS, callbacks=[callbacks, early_stopping])

      # results directory
      RES_DIR = f'/content/drive/MyDrive/Colab Notebooks/Final project/Experiments/results/encoder/{net}'
      if not os.path.exists(RES_DIR):
          os.makedirs(RES_DIR)

      # summarize history for accuracy
      fig = plt.figure()
      plt.plot(history.history['sparse_categorical_accuracy'])
      plt.plot(history.history['val_sparse_categorical_accuracy'])
      plt.title(f'{net}: {heads} Heads, {layers} layers\n accuracy')
      plt.ylabel('accuracy')
      plt.xlabel('epoch')
      plt.legend(['train', 'val'], loc='upper left')
      plt.show()
      fig.savefig(RES_DIR + f'/net: {net} numHeads: {heads} layers: {layers} accuracy', dpi=fig.dpi)

      # summarize history for loss
      fig = plt.figure()
      plt.plot(history.history['loss'])
      plt.plot(history.history['val_loss'])
      plt.title(f'{net}: {heads} Heads, {layers} layers\n loss')
      plt.ylabel('loss')
      plt.xlabel('epoch')
      plt.legend(['train', 'val'], loc='upper left')
      plt.show()
      fig.savefig(RES_DIR + f'/net: {net} numHeads: {heads} layers: {layers} loss', dpi=fig.dpi)

      ## val Cmat
      cm, acc = GetCmat(myModel, networksDict[net]['val'], y_val)
      fig = plt.figure()
      disp = ConfusionMatrixDisplay(confusion_matrix=cm)
      disp.plot()
      plt.title(f'{net}: {heads} Heads, {layers} layers\n val acc: {acc:.5}')
      plt.savefig(RES_DIR + f'/net: {net} numHeads: {heads} layers: {layers} Cmat val', dpi=fig.dpi)
      plt.show()

      ## test Cmat
      cm, acc = GetCmat(myModel, networksDict[net]['test'], y_test)
      fig = plt.figure()
      disp = ConfusionMatrixDisplay(confusion_matrix=cm)
      disp.plot()
      plt.title(f'{net}: {heads} Heads, {layers} layers\n test acc: {acc:.5}')
      plt.savefig(RES_DIR + f'/net: {net} numHeads: {heads} layers: {layers} Cmat test', dpi=fig.dpi)
      plt.show()


      results, results_prob = Get_Results(args, myModel, networksDict[net]['train'], y_train, train_list, train_len, networksDict[net]['test'], y_test, test_list, test_len, clip_time)

      myModel.save(f'/content/drive/MyDrive/Colab Notebooks/Final project/Experiments/models/encoder_models/{net}/Model_{net}_numHeads_{heads}_num_layers_{layers}')

      res_path = (RES_DIR + 
                  '/%s_%d_net_%s' %(args.roi_name, args.roi, net) +
                  '_k_layers_%d' %(layers) +
                  '_heads_%d_batch_size_%d' %(heads, BATCH_SIZE) +
                  '_num_epochs_%d.pkl' %(EPOCHS))

      # save results
      with open(res_path, 'wb') as f:
          pickle.dump([results, results_prob], f)

In [ ]:
### offset and shuffled

import pickle
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay

def make_batches(ds):
  return (
      ds
      .cache()
      .shuffle(BUFFER_SIZE)
      .batch(BATCH_SIZE)
      .prefetch(tf.data.AUTOTUNE))

def GetCmat(model, X, y):
  y_hat_hold = model.predict(X)
  y_hat = np.argmax(y_hat_hold, axis=2)
  true_y = y

  y_overtime = []
  y_hat_overtime = []
  for rows_y,rows_y_hat in zip(true_y,y_hat):
    values, counts = np.unique(rows_y, return_counts=True)
    ind = np.argmax(counts)
    if values[ind] == 15:
      counts[ind] = 0
      ind = np.argmax(counts)
    y_overtime.append(values[ind])

    values, counts = np.unique(rows_y_hat, return_counts=True)
    ind = np.argmax(counts)
    if values[ind] == 15:
      counts[ind] = 0
      ind = np.argmax(counts)
    y_hat_overtime.append(values[ind])

  cm = confusion_matrix(y_overtime,y_hat_overtime)
  acc = accuracy_score(y_overtime,y_hat_overtime)
  return cm, acc

def printLoop(net, head, layer):
  print('-----------------------------------------------------------------------------------------------------')
  print('-----------------------------------------------------------------------------------------------------')
  print('-----------------------------------------------------------------------------------------------------')
  print(f'---------------------------NET: {net}  number of heads: {head} number of layers: {layer}--------------------------')
  print('-----------------------------------------------------------------------------------------------------')
  print('-----------------------------------------------------------------------------------------------------')
  print('-----------------------------------------------------------------------------------------------------')

def ShuffleAndOffset(y):
  y_shuffle = tf.random.shuffle(y)
  y_offset = np.array(y)
  for i,_ in enumerate(y):
    j = np.where(y[i,:]>0)
    y_offset[i,j] += 2
    j = np.where(y_offset[i,:]==15)
    y_offset[i,j] = 1
    j = np.where(y_offset[i,:]==16)
    y_offset[i,j] = 2
    j = np.where(y_offset[i,:]==17)
    y_offset[i,j] = 15
  y_offset = tf.convert_to_tensor(y_offset)
  return y_shuffle, y_offset

y_shuffled_t, y_offset_t = ShuffleAndOffset(y_train)

# num_layers = [6]
# num_heads = [1]

for net, y_train_copy in zip(['fullOffset'], [y_offset_t]): # in zip(['fullShuffled', 'fullOffset'], [y_shuffled_t, y_offset_t]):
  for heads in num_heads:
    for layers in num_layers:
      printLoop(net, heads, layers)
      train_data = tf.data.Dataset.from_tensor_slices((networksDict[net]['train'],y_train_copy))
      val_data = tf.data.Dataset.from_tensor_slices((networksDict[net]['val'],y_val))
      input = networksDict[net]['train']
      train_batch = make_batches(train_data)
      val_batch = make_batches(val_data)

      myModel = ModelTrunk(inputs=input, name=f'Model_{net}_numHeads_{heads}', time2vec_dim=1, num_heads=heads, head_size=key_dim, ff_dim=dff, num_layers=layers, dropout=dropout_rate, classes=(len(clip_time)+1))

      myModel.compile(
          optimizer=keras.optimizers.Adam(learning_rate=1e-4),
          loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
          metrics=[keras.metrics.SparseCategoricalAccuracy()]
      )

      callbacks = keras.callbacks.LearningRateScheduler(lr_scheduler, verbose=0)

      # EarlyStopping criteria
      early_stopping = keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)

      # history = myModel.fit(train_batch, validation_data=val_batch, batch_size=BATCH_SIZE, epochs=EPOCHS, callbacks=[callbacks, early_stopping])
      history = myModel.fit(train_batch, batch_size=BATCH_SIZE, epochs=EPOCHS, callbacks=[callbacks]) # without val for offset

      # results directory
      RES_DIR = f'/content/drive/MyDrive/Colab Notebooks/Final project/Experiments/results/encoder/{net}'
      if not os.path.exists(RES_DIR):
          os.makedirs(RES_DIR)

      # summarize history for accuracy
      fig = plt.figure()
      plt.plot(history.history['sparse_categorical_accuracy'])
      # plt.plot(history.history['val_sparse_categorical_accuracy'])
      plt.title(f'{net}: {heads} Heads, {layers} layers\n accuracy')
      plt.ylabel('accuracy')
      plt.xlabel('epoch')
      # plt.legend(['train', 'val'], loc='upper left')
      plt.show()
      fig.savefig(RES_DIR + f'/net: {net} numHeads: {heads} layers: {layers} accuracy', dpi=fig.dpi)

      # summarize history for loss
      fig = plt.figure()
      plt.plot(history.history['loss'])
      # plt.plot(history.history['val_loss'])
      plt.title(f'{net}: {heads} Heads, {layers} layers\n loss')
      plt.ylabel('loss')
      plt.xlabel('epoch')
      # plt.legend(['train', 'val'], loc='upper left')
      plt.show()
      fig.savefig(RES_DIR + f'/net: {net} numHeads: {heads} layers: {layers} loss', dpi=fig.dpi)

      ## val Cmat
      cm, acc = GetCmat(myModel, networksDict[net]['val'], y_val)
      fig = plt.figure()
      disp = ConfusionMatrixDisplay(confusion_matrix=cm)
      disp.plot()
      plt.title(f'{net}: {heads} Heads, {layers} layers\n val acc: {acc:.5}')
      plt.savefig(RES_DIR + f'/net: {net} numHeads: {heads} layers: {layers} Cmat val', dpi=fig.dpi)
      plt.show()

      ## test Cmat
      cm, acc = GetCmat(myModel, networksDict[net]['test'], y_test)
      fig = plt.figure()
      disp = ConfusionMatrixDisplay(confusion_matrix=cm)
      disp.plot()
      plt.title(f'{net}: {heads} Heads, {layers} layers\n test acc: {acc:.5}')
      plt.savefig(RES_DIR + f'/net: {net} numHeads: {heads} layers: {layers} Cmat test', dpi=fig.dpi)
      plt.show()


      results, results_prob = Get_Results(args, myModel, networksDict[net]['train'], y_train, train_list, train_len, networksDict[net]['test'], y_test, test_list, test_len, clip_time)

      myModel.save(f'/content/drive/MyDrive/Colab Notebooks/Final project/Experiments/models/encoder_models/{net}/Model_{net}_numHeads_{heads}_num_layers_{layers}')

      res_path = (RES_DIR + 
                  '/%s_%d_net_%s' %(args.roi_name, args.roi, net) +
                  '_k_layers_%d' %(layers) +
                  '_heads_%d_batch_size_%d' %(heads, BATCH_SIZE) +
                  '_num_epochs_%d.pkl' %(EPOCHS))

      # save results
      with open(res_path, 'wb') as f:
          pickle.dump([results, results_prob], f)

In [ ]:
### shuffle statistics

import pickle
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay

def make_batches(ds):
  return (
      ds
      .cache()
      .shuffle(BUFFER_SIZE)
      .batch(BATCH_SIZE)
      .prefetch(tf.data.AUTOTUNE))

def GetCmat(model, X, y):
  y_hat_hold = model.predict(X)
  y_hat = np.argmax(y_hat_hold, axis=2)
  true_y = y

  y_overtime = []
  y_hat_overtime = []
  for rows_y,rows_y_hat in zip(true_y,y_hat):
    values, counts = np.unique(rows_y, return_counts=True)
    ind = np.argmax(counts)
    if values[ind] == 15:
      counts[ind] = 0
      ind = np.argmax(counts)
    y_overtime.append(values[ind])

    values, counts = np.unique(rows_y_hat, return_counts=True)
    ind = np.argmax(counts)
    if values[ind] == 15:
      counts[ind] = 0
      ind = np.argmax(counts)
    y_hat_overtime.append(values[ind])

  cm = confusion_matrix(y_overtime,y_hat_overtime)
  acc = accuracy_score(y_overtime,y_hat_overtime)
  return cm, acc

def printLoop(net, head, layer, i):
  print('-----------------------------------------------------------------------------------------------------')
  print('-----------------------------------------------------------------------------------------------------')
  print('-----------------------------------------------------------------------------------------------------')
  print(f'---------------------------NET: {net}  number of heads: {head} number of layers: {layer} round: {i}--------------------------')
  print('-----------------------------------------------------------------------------------------------------')
  print('-----------------------------------------------------------------------------------------------------')
  print('-----------------------------------------------------------------------------------------------------')

def ShuffleAndOffset(y):
  y_shuffle = tf.random.shuffle(y)
  y_offset = np.array(y)
  for i,_ in enumerate(y):
    j = np.where(y[i,:]>0)
    y_offset[i,j] += 2
    j = np.where(y_offset[i,:]==15)
    y_offset[i,j] = 1
    j = np.where(y_offset[i,:]==16)
    y_offset[i,j] = 2
    j = np.where(y_offset[i,:]==17)
    y_offset[i,j] = 15
  y_offset = tf.convert_to_tensor(y_offset)
  return y_shuffle, y_offset

y_shuffled_t, y_offset_t = ShuffleAndOffset(y_train)

num_layers = [6]
num_heads = [1]
num_of_samples = 10

shuffled_acc = np.zeros((num_of_samples,1))


for net, y_train_copy in zip(['fullShuffled'], [y_shuffled_t]): # in zip(['fullShuffled', 'fullOffset'], [y_shuffled_t, y_offset_t]):
  for heads in num_heads:
    for layers in num_layers:
      for i in range(0,num_of_samples):

        printLoop(net, heads, layers, i)

        train_data = tf.data.Dataset.from_tensor_slices((networksDict[net]['train'],y_train_copy))
        val_data = tf.data.Dataset.from_tensor_slices((networksDict[net]['val'],y_val))
        input = networksDict[net]['train']
        train_batch = make_batches(train_data)
        val_batch = make_batches(val_data)

        myModel = ModelTrunk(inputs=input, name=f'Model_{net}_numHeads_{heads}', time2vec_dim=1, num_heads=heads, head_size=key_dim, ff_dim=dff, num_layers=layers, dropout=dropout_rate, classes=(len(clip_time)+1))

        myModel.compile(
            optimizer=keras.optimizers.Adam(learning_rate=1e-4),
            loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
            metrics=[keras.metrics.SparseCategoricalAccuracy()]
        )
        callbacks = keras.callbacks.LearningRateScheduler(lr_scheduler, verbose=0)

        # EarlyStopping criteria
        early_stopping = keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)

        history = myModel.fit(train_batch, validation_data=val_batch, batch_size=BATCH_SIZE, epochs=EPOCHS, callbacks=[callbacks, early_stopping])

        # results directory
        RES_DIR = f'/content/drive/MyDrive/Colab Notebooks/Final project/Experiments/results/encoder/{net}'
        if not os.path.exists(RES_DIR):
            os.makedirs(RES_DIR)

        # summarize history for accuracy
        fig = plt.figure()
        plt.plot(history.history['sparse_categorical_accuracy'])
        # plt.plot(history.history['val_sparse_categorical_accuracy'])
        plt.title(f'{net}: {heads} Heads, {layers} layers\n accuracy')
        plt.ylabel('accuracy')
        plt.xlabel('epoch')
        # plt.legend(['train', 'val'], loc='upper left')
        plt.show()
        # fig.savefig(RES_DIR + f'/net: {net} numHeads: {heads} layers: {layers} accuracy', dpi=fig.dpi)

        # summarize history for loss
        fig = plt.figure()
        plt.plot(history.history['loss'])
        # plt.plot(history.history['val_loss'])
        plt.title(f'{net}: {heads} Heads, {layers} layers\n loss')
        plt.ylabel('loss')
        plt.xlabel('epoch')
        # plt.legend(['train', 'val'], loc='upper left')
        plt.show()
        # fig.savefig(RES_DIR + f'/net: {net} numHeads: {heads} layers: {layers} loss', dpi=fig.dpi)

        ## val Cmat
        cm, acc = GetCmat(myModel, networksDict[net]['val'], y_val)
        fig = plt.figure()
        disp = ConfusionMatrixDisplay(confusion_matrix=cm)
        disp.plot()
        plt.title(f'{net}: {heads} Heads, {layers} layers\n val acc: {acc:.5}')
        # plt.savefig(RES_DIR + f'/net: {net} numHeads: {heads} layers: {layers} Cmat val', dpi=fig.dpi)
        plt.show()

        ## test Cmat
        cm, acc = GetCmat(myModel, networksDict[net]['test'], y_test)
        fig = plt.figure()
        disp = ConfusionMatrixDisplay(confusion_matrix=cm)
        disp.plot()
        plt.title(f'{net}: {heads} Heads, {layers} layers\n test acc: {acc:.5}')
        # plt.savefig(RES_DIR + f'/net: {net} numHeads: {heads} layers: {layers} Cmat test', dpi=fig.dpi)
        plt.show()
        shuffled_acc[i] = acc

        # results, results_prob = Get_Results(args, myModel, networksDict[net]['train'], y_train, train_list, train_len, networksDict[net]['test'], y_test, test_list, test_len, clip_time)

        # myModel.save(f'/content/drive/MyDrive/Colab Notebooks/Final project/Experiments/models/encoder_models/{net}/Model_{net}_numHeads_{heads}_num_layers_{layers}')
        
      res_path = (RES_DIR + 
                  '/%s_%d_net_%s' %(args.roi_name, args.roi, net) +
                  '_k_layers_%d' %(layers) +
                  '_heads_%d_batch_size_%d' %(heads, BATCH_SIZE) +
                  '_num_epochs_%d_shuffled_statistics.npy' %(EPOCHS))

      # save results
      np.save(res_path, shuffled_acc)